In [34]:
import os
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [35]:
# reading all images from directory
Images = {}
Folder = 'Grouped_Output'
for filename in os.listdir(Folder):
    # opening all folders in this directory and saving the images in a dictionary
    Images[filename] = []
    for img in os.listdir(Folder + '/' + filename):
        Images[filename].append(img)

In [36]:
# # printing one image from each folder
# for folder in Images.keys():
#     img = Image.open(Folder + '/' + folder + '/' + Images[folder][0])
#     plt.imshow(np.array(img))
#     plt.show()

In [37]:
# Pre-processing the images 
# 1. Resizing the images to 224x224
# 2. Normalizing the images
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

In [38]:
# applying transformation to all images
Images_tensor = {}
for folder in Images.keys():
    Images_tensor[folder] = []
    for img in Images[folder]:
        img = Image.open(Folder + '/' + folder + '/' + img)
        img = transform(img)
        Images_tensor[folder].append(img)

## VAE Implementation:  
1. Define the encoder and decoder networks.
2. Define the VAE loss function (reconstruction loss + KL divergence).
3. Train the VAE on the augmented dataset

In [45]:

class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(224*224*3, 400) 
        self.fc2_mu = nn.Linear(400, latent_dim)
        self.fc2_logvar = nn.Linear(400, latent_dim)

    def forward(self, x):
        x = x.view(-1, 224*224*3) 
        x = F.relu(self.fc1(x))
        mu = self.fc2_mu(x)
        logvar = self.fc2_logvar(x)
        return mu, logvar

In [46]:
class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 400)
        self.fc2 = nn.Linear(400, 224*224*3)

    def forward(self, z):
        z = F.relu(self.fc1(z))
        z = torch.sigmoid(self.fc2(z))
        z = z.view(-1, 3, 224, 224)
        return z

In [41]:
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [47]:
# Training the VAE
vae = VAE(latent_dim=20)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

epoch_Number = 10
for epoch in range(epoch_Number):
    for folder in Images_tensor.keys():
        for img in Images_tensor[folder]:
            img = img.view(-1, 3, 224, 224)  # Ensure the image is in the correct shape
            recon_img, mu, logvar = vae(img)
            loss = vae_loss(recon_img, img, mu, logvar)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

KeyboardInterrupt: 

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(100, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 64*64)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(64*64, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = torch.sigmoid(self.fc4(x))
        return x

# Initialize models
generator = Generator()
discriminator = Discriminator()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

# Loss function
adversarial_loss = nn.BCELoss()

In [ ]:
# Training
epoch_Number = 10
for epoch in range(epoch_Number):
    for folder in Images_tensor.keys():
        for img in Images_tensor[folder]:
            img = img.view(-1, 64*64)
            z = torch.randn(1, 100)
            fake_img = generator(z)
            real_label = torch.ones(1, 1)
            fake_label = torch.zeros(1, 1)

            # Train Discriminator
            optimizer_D.zero_grad()
            real_output = discriminator(img)
            fake_output = discriminator(fake_img)
            real_loss = adversarial_loss(real_output, real_label)
            fake_loss = adversarial_loss(fake_output, fake_label)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_D.step()

            # Train Generator
            optimizer_G.zero_grad()
            z = torch.randn(1, 100)
            fake_img = generator(z)
            output = discriminator(fake_img)
            g_loss = adversarial_loss(output, real_label)
            g_loss.backward()
            optimizer_G.step()
    print(f'Epoch: {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

In [ ]:
# Generating images
z = torch.randn(1, 100)
fake_img = generator(z)
fake_img = fake_img.view(64, 64)
plt.imshow(fake_img.detach().numpy())
plt.show()